In [1]:
import sionna
import tensorflow as tf

# Import Sionna RT components
from sionna.rt import load_scene, PlanarArray, Receiver, Transmitter

2023-06-21 14:17:47.023932: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-21 14:17:47.104650: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-21 14:17:47.520146: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.6/lib64
2023-06-21 14:17:47.520182: W tensorflow/compiler/xla/stream

In [2]:
seed = 1234

# Load integrated scene
scene = load_scene(sionna.rt.scene.simple_street_canyon) # Try also sionna.rt.scene.etoile                              polarization="cross")

# Configure antenna array for all transmitters
scene.tx_array = PlanarArray(num_rows=1, 
                             num_cols=1,
                             vertical_spacing=0.5,
                             horizontal_spacing=0.5,
                             pattern="iso",
                             polarization="cross")

# Configure antenna array for all receivers
scene.rx_array = PlanarArray(num_rows=1,
                             num_cols=1,
                             vertical_spacing=0.5,
                             horizontal_spacing=0.5,
                             pattern="iso",
                             polarization="cross")


n_sources = 100
n_targets = 100

min_x, max_x = -100, 100
min_y, max_y = -100, 100
min_z, max_z = 0, 100

for i, x, y, z in zip(
    range(n_sources),
    tf.random.uniform((n_sources,), minval=min_x, maxval=max_x, seed=seed),
    tf.random.uniform((n_sources,), minval=min_y, maxval=max_y, seed=seed),
    tf.random.uniform((n_sources,), minval=min_z, maxval=max_z, seed=seed),
):
    # Create transmitter
    tx = Transmitter(name=f"tx_{i}",
                     position=[x, y, z])

    # Add transmitter instance to scene
    scene.add(tx)
    
for i, x, y, z in zip(
    range(n_targets),
    tf.random.uniform((n_targets,), minval=min_x, maxval=max_x, seed=seed),
    tf.random.uniform((n_targets,), minval=min_y, maxval=max_y, seed=seed),
    tf.random.uniform((n_targets,), minval=min_z, maxval=max_z, seed=seed),
):
    # Create receiver
    rx = Receiver(name=f"rx_{i}",
                  position=[x, y, z])

    # Add receiver instance to scene
    scene.add(rx)


scene.preview()

2023-06-21 14:17:48.324315: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Renderer(camera=PerspectiveCamera(aspect=1.31, children=(DirectionalLight(intensity=0.25, position=(0.0, 0.0, …

In [3]:
# Compute propagation paths
paths = scene.compute_paths(max_depth=2,
                            method="exhaustive",
                            #method="stochastic", # For small scenes the method can be also set to "exhaustive"
                            num_samples=1e6,     # Number of rays shot into random directions, too few rays can lead to missing paths
                            seed=seed)           # By fixing the seed, reproducible results can be ensured

scene.preview(paths, show_devices=True, show_paths=True) # Use the mouse to focus on the visualized paths

Renderer(camera=PerspectiveCamera(aspect=1.31, children=(DirectionalLight(intensity=0.25, position=(0.0, 0.0, …

In [4]:
# [max_depth, num_targets, num_sources, max_num_paths, 3 (x, y, z)]
# max_num_paths cannot be predicted, but is always <= num_primitives**max_depth + 1
paths.vertices

<tf.Tensor: shape=(2, 100, 100, 8, 3), dtype=float32, numpy=
array([[[[[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
          [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
          [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
          ...,
          [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
          [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
          [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],

         [[-6.21076508e+01, -5.96427689e+01,  4.26178589e+01],
          [-3.15986156e+01, -4.43166313e+01, -3.07960510e-02],
          [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
          ...,
          [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
          [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
          [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],

         [[-6.21076508e+01,  2.15140343e+01,  3.59053116e+01],
          [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
          [ 0.00000000e